# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [3]:
# Your code here
cur.execute("""SELECT * 
               FROM employees e
               JOIN offices o
               ON e.officeCode = o.officeCode""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC),6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA),4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [9]:
# Your code here
cur.execute("""SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS n_employees
               FROM offices AS o
               LEFT JOIN employees AS e
               USING(officeCode)
               GROUP BY officeCode
               HAVING n_employees = 0;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,officeCode,city,n_employees
0,27,Boston,0


## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold


In [12]:
# Your code here What is the highest selling product?
cur.execute("""SELECT productCode, productName, SUM(quantityOrdered) AS totQTY
               FROM products p
               LEFT JOIN orderdetails o
               USING(productCode)
               GROUP BY productCode
               ORDER BY totQTY DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,productCode,productName,totQTY
0,S18_3232,1992 Ferrari 360 Spider red,1808.0
1,S18_1342,1937 Lincoln Berline,1111.0
2,S700_4002,American Airlines: MD-11S,1085.0
3,S18_3856,1941 Chevrolet Special Deluxe Cabriolet,1076.0
4,S50_1341,1930 Buick Marquette Phaeton,1074.0


In [15]:
# Your code here What is the lowest selling product?
cur.execute("""SELECT productCode, productName, SUM(quantityOrdered) AS totQTY
               FROM products p
               LEFT JOIN orderdetails o
               USING(productCode)
               GROUP BY productCode
               HAVING totQTY NOT NULL
               ORDER BY totQTY;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.tail()


,productCode,productName,totQTY
104,S50_1341,1930 Buick Marquette Phaeton,1074
105,S18_3856,1941 Chevrolet Special Deluxe Cabriolet,1076
106,S700_4002,American Airlines: MD-11S,1085
107,S18_1342,1937 Lincoln Berline,1111
108,S18_3232,1992 Ferrari 360 Spider red,1808


In [21]:
# Your code here MSRP vs Price
cur.execute("""SELECT productName, AVG(MSRP), priceEach
               FROM products
               LEFT JOIN orderdetails
               USING(productCode)
               GROUP BY productName""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.tail()


,productName,AVG(MSRP),priceEach
105,The Mayflower,86.61,69.29
106,The Queen Mary,99.31,80.44
107,The Schooner Bluenose,66.67,53.34
108,The Titanic,100.17,83.14
109,The USS Constitution Ship,72.28,57.82


## Level Up: Display the names of every individual product that each employee has sold

In [25]:
# Your code here
cur.execute("""SELECT firstName, lastName, productName
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang


## Level Up: Display the Number of Products each employee has sold

In [27]:
# Your code here
cur.execute("""SELECT firstName, lastName, COUNT(productName)
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products
               USING(productCode)
               GROUP BY lastName
               ORDER BY firstName""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,firstName,lastName,COUNT(productName)
0,Andy,Fixter,185
1,Barry,Jones,220
2,Foon Yue,Tseng,142
3,George,Vanauf,211
4,Gerard,Hernandez,396
5,Julie,Firrelli,124
6,Larry,Bott,236
7,Leslie,Jennings,331
8,Leslie,Thompson,114
9,Loui,Bondur,177


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!